# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'11-22-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'11-22-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-11-23 05:25:26,33.93911,67.709953,44706,1687,35934,7085.0,Afghanistan,114.841781,3.773543
1,NaN,NaN,NaN,Albania,2020-11-23 05:25:26,41.15330,20.168300,32761,699,15842,16220.0,Albania,1138.404337,2.133635
2,NaN,NaN,NaN,Algeria,2020-11-23 05:25:26,28.03390,1.659600,74862,2272,48794,23796.0,Algeria,170.718858,3.034918
3,NaN,NaN,NaN,Andorra,2020-11-23 05:25:26,42.50630,1.521800,6256,76,5358,822.0,Andorra,8096.809681,1.214834
4,NaN,NaN,NaN,Angola,2020-11-23 05:25:26,-11.20270,17.873900,14493,337,7346,6810.0,Angola,44.096884,2.325260


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,42969,43035,43240,43403,43628,43851,44228,44443,44503,44706
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,26701,27233,27830,28432,29126,29837,30623,31459,32196,32761
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,65975,66819,67679,68589,69591,70629,71652,72755,73774,74862


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1595,1605,1617,1626,1638,1645,1650,1661,1675,1687
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,605,612,623,631,637,646,657,672,685,699
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2124,2139,2154,2168,2186,2206,2224,2236,2255,2272


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35036,35067,35092,35137,35160,35295,35350,35370,35422,35934
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,12667,12767,12889,13453,13804,14216,14565,15055,15469,15842
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,43779,44199,44633,45148,45148,46326,46962,47581,48183,48794


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2417,2435,2456,2481,2506,2529,2554,2580,2597,2617
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,7523,7596,7646,7696,7772,7849,7933,8038,8131,8199
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1117,1123,1128,1130,1134,1137,1145,1151,1157,1160


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,36,36,36,36,37,37,39,39,39,39
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,84,84,84,84,84,84,84,84,84,84
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,10,10,10,10,10


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
646,1001.0,Autauga,Alabama,US,2020-11-23 05:25:26,32.539527,-86.644082,2617,39,0,2578.0,"Autauga, Alabama, US",4684.171902,1.490256
683,1075.0,Lamar,Alabama,US,2020-11-23 05:25:26,33.779950,-88.096680,622,7,0,615.0,"Lamar, Alabama, US",4505.613908,1.125402
684,1077.0,Lauderdale,Alabama,US,2020-11-23 05:25:26,34.901719,-87.656247,3783,53,0,3730.0,"Lauderdale, Alabama, US",4079.629889,1.401004


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,12246849,256782,4526513
India,9139865,133738,8562641
Brazil,6071401,169183,5457973


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,12246849,256782,4526513,7463554,2020-11-23 05:25:26,37.936303,-91.379001
India,9139865,133738,8562641,443486,2020-11-23 05:25:26,23.088275,81.806127
Brazil,6071401,169183,5457973,444245,2020-11-23 05:25:26,-12.669522,-48.480493
France,2191180,48807,157259,1985114,2020-11-23 05:25:26,8.314863,-21.695174
Russia,2071858,35838,1582768,453252,2020-11-23 05:25:26,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1153612,21013,0
California,1114524,18726,0
Florida,938414,17991,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1153612,21013,0,1132599,2020-11-23 05:25:26,31.660643,-98.653069
California,1114524,18726,0,1095798,2020-11-23 05:25:26,37.843962,-120.728594
Florida,938414,17991,0,920423,2020-11-23 05:25:26,28.940755,-82.700744
Illinois,656298,12050,0,644248,2020-11-23 05:25:26,39.843618,-89.178330
New York,596214,34319,0,561895,2020-11-23 05:25:26,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,364520,7438,0
Illinois,Cook,280896,6303,0
Florida,Miami-Dade,214943,3766,0
Arizona,Maricopa,190034,3896,0
Texas,Harris,182192,2965,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,364520,7438,0,357082,2020-11-23 05:25:26,34.308284,-118.228241,6037.0
Illinois,Cook,280896,6303,0,274593,2020-11-23 05:25:26,41.841448,-87.816588,17031.0
Florida,Miami-Dade,214943,3766,0,211177,2020-11-23 05:25:26,25.611236,-80.551706,12086.0
Arizona,Maricopa,190034,3896,0,186138,2020-11-23 05:25:26,33.348359,-112.491815,4013.0
Texas,Harris,182192,2965,0,179227,2020-11-23 05:25:26,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,44443,31459,72755,6142,14267,139,1359042,123646,27806,235351,...,4477,71208,1,99017,1305,68768,10,2090,17373,9120
2020-11-21,44503,32196,73774,6207,14413,139,1366182,124839,27821,241962,...,4564,71431,1,99435,1306,70254,10,2093,17394,9172
2020-11-22,44706,32761,74862,6256,14493,139,1370366,126224,27834,247188,...,4699,71617,1,99835,1307,71644,10,2099,17424,9220


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,1661,672,2236,76,334,4,36790,1900,907,2224,...,69,603,0,866,35,606,1,608,356,265
2020-11-21,1675,685,2255,76,336,4,36902,1931,907,2328,...,69,603,0,869,35,620,1,608,356,265
2020-11-22,1687,699,2272,76,337,4,37002,1952,907,2388,...,71,603,0,871,35,636,1,608,357,268


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,35370,15055,47581,5239,7117,127,1177819,90754,25521,156483,...,3547,68451,0,94046,1142,57879,8,1434,16606,8229
2020-11-21,35422,15469,48183,5290,7273,128,1187053,92829,25522,162751,...,3621,68659,0,94355,1142,58383,8,1441,16659,8235
2020-11-22,35934,15842,48794,5358,7346,128,1195492,94090,25527,169260,...,3679,68827,0,94355,1142,59021,8,1448,16701,8250


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2417,2435,2456,2481,2506,2529,2554,2580,2597,2617
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,579,581,584,584,588,596,600,601,609,622
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3221,3259,3298,3360,3431,3487,3578,3653,3712,3783
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1050,1071,1076,1080,1098,1124,1152,1179,1201,1214
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7122,7181,7212,7240,7278,7331,7363,7421,7478,7504


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,228373,26544,0,291696,141916,1089750,188566,101469,30816,49,...,331532,1132690,170584,3459,1482,213331,139543,38480,365190,27129
2020-11-21,230708,27224,0,295334,143821,1105435,194679,101469,31321,49,...,335887,1144509,173979,3546,1491,215679,141260,39598,372219,27410
2020-11-22,232506,27773,0,299665,145173,1114524,198600,101469,31808,49,...,340476,1153612,177176,3629,1504,217796,141260,40478,376238,28169


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-20        2580    8038    1151  1024   2704     684   1120    5896   
2020-11-21        2597    8131    1157  1036   2735     688   1132    5924   
2020-11-22        2617    8199    1160  1136   2754     689   1133    5964   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-20         1663      959  ...    1158    282     1592      349   
2020-11-21         1669      979  ...    1161    284     1595      369   
2020-11-22         1675      985  ...    1168    291     1652      374   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-20           1353  1336   894          0      290    364  
2020-11-21           1378  1360   901          0      298    366  
2020-11-22           1421  1371   920          0      315    372  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,3451,101,0,6427,2321,18640,2745,4828,746,0,...,4202,20751,773,62,23,3912,2619,639,3091,176
2020-11-21,3457,102,0,6457,2337,18672,2787,4828,746,0,...,4211,20903,787,63,23,3938,2619,658,3143,176
2020-11-22,3457,103,0,6464,2357,18726,2806,4828,748,0,...,4266,21013,793,63,23,3938,2619,662,3150,176


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-20          39      84      10   18     35      19     41     113   
2020-11-21          39      84      10   17     36      19     41     113   
2020-11-22          39      84      10   17     36      19     41     113   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-20           48       24  ...       4      5       10        1   
2020-11-21           48       24  ...       4      5       10        1   
2020-11-22           48       24  ...       4      5       10        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-20              6     2     4          0        7      0  
2020-11-21              6     2     4          0        7      0  
2020-11-22              6     2     4          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.004861,0.027300,0.015394,0.012529,0.009410,0.0,0.007120,0.013666,0.000504,0.029158,...,0.022847,0.004047,0.0,0.003568,0.000767,0.021874,0.0,0.001918,0.001326,0.008180
2020-11-21,0.001350,0.023427,0.014006,0.010583,0.010233,0.0,0.005254,0.009649,0.000539,0.028090,...,0.019433,0.003132,0.0,0.004221,0.000766,0.021609,0.0,0.001435,0.001209,0.005702
2020-11-22,0.004561,0.017549,0.014748,0.007894,0.005551,0.0,0.003063,0.011094,0.000467,0.021598,...,0.029579,0.002604,0.0,0.004023,0.000766,0.019785,0.0,0.002867,0.001725,0.005233


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.006667,0.022831,0.005396,0.0,0.003003,0.0,0.007062,0.016043,0.0,0.051040,...,0.000000,0.003328,NaN,0.003476,0.0,0.013378,0.0,0.0,0.000000,0.000000
2020-11-21,0.008429,0.019345,0.008497,0.0,0.005988,0.0,0.003044,0.016316,0.0,0.046763,...,0.000000,0.000000,NaN,0.003464,0.0,0.023102,0.0,0.0,0.000000,0.000000
2020-11-22,0.007164,0.020438,0.007539,0.0,0.002976,0.0,0.002710,0.010875,0.0,0.025773,...,0.028986,0.000000,NaN,0.002301,0.0,0.025806,0.0,0.0,0.002809,0.011321


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.000566,0.033642,0.013181,0.021845,0.007788,0.000000,0.008826,0.026768,0.000392,0.046422,...,0.019546,0.004815,NaN,0.003789,0.0,0.007134,0.0,0.004202,0.000904,0.004149
2020-11-21,0.001470,0.027499,0.012652,0.009735,0.021919,0.007874,0.007840,0.022864,0.000039,0.040055,...,0.020863,0.003039,NaN,0.003286,0.0,0.008708,0.0,0.004881,0.003192,0.000729
2020-11-22,0.014454,0.024113,0.012681,0.012854,0.010037,0.000000,0.007109,0.013584,0.000196,0.039994,...,0.016018,0.002447,NaN,0.000000,0.0,0.010928,0.0,0.004858,0.002521,0.001821


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,0.010903,0.017752,NaN,0.015566,0.014737,0.011675,0.031537,0.02101,0.021988,0.0,...,0.010497,0.011563,0.027639,0.045015,0.008850,0.012069,0.015515,0.028905,0.019762,0.036685
2020-11-21,0.010225,0.025618,NaN,0.012472,0.013423,0.014393,0.032418,0.00000,0.016388,0.0,...,0.013136,0.010434,0.019902,0.025152,0.006073,0.011006,0.012304,0.029054,0.019248,0.010358
2020-11-22,0.007793,0.020166,NaN,0.014665,0.009401,0.008222,0.020141,0.00000,0.015549,0.0,...,0.013662,0.007954,0.018376,0.023407,0.008719,0.009816,0.000000,0.022223,0.010797,0.027691


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-20      0.010180  0.013236  0.005240  0.012859  0.007827  0.005882   
2020-11-21      0.006589  0.011570  0.005213  0.011719  0.011464  0.005848   
2020-11-22      0.007701  0.008363  0.002593  0.096525  0.006947  0.001453   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-20      0.006289  0.014104  0.013406  0.003138  ...  0.016681   
2020-11-21      0.010714  0.004749  0.003608  0.020855  ...  0.002591   
2020-11-22      0.000883  0.006752  0.003595  0.006129  ...  0.006029   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-20      0.007143  0.071332  0.041791   0.066194  0.029276  0.028769   
2020-11-21      0.007092  0.001884  0.057307   0.018477  0.017964  0.007830   
2020-11-22      0.024648  0.035737  0.013550   0.031205  0.008088  0.021088   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-20            NaN  0.058394  0.025352  
2020-11-21            NaN  0.027586  0.005495  
2020-11-22            NaN  0.057047  0.016393  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,0.009359,0.000000,NaN,0.006736,0.010448,0.005285,0.005495,0.004787,0.005391,NaN,...,0.017926,0.009044,0.022487,0.016393,0.0,0.004107,0.006147,0.025682,0.026910,0.0
2020-11-21,0.001739,0.009901,NaN,0.004668,0.006894,0.001717,0.015301,0.000000,0.000000,NaN,...,0.002142,0.007325,0.018111,0.016129,0.0,0.006646,0.000000,0.029734,0.016823,0.0
2020-11-22,0.000000,0.009804,NaN,0.001084,0.008558,0.002892,0.006817,0.000000,0.002681,NaN,...,0.013061,0.005262,0.007624,0.000000,0.0,0.000000,0.000000,0.006079,0.002227,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour      Bibb    Blount Bullock Butler   
2020-11-20         0.0     0.0     0.0  0.000000  0.000000     0.0    0.0   
2020-11-21         0.0     0.0     0.0 -0.055556  0.028571     0.0    0.0   
2020-11-22         0.0     0.0     0.0  0.000000  0.000000     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-11-20      0.018018      0.0      0.0  ...     0.0    0.0      0.0   
2020-11-21      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2020-11-22      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-20          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-21          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-11-22          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.005778,0.026132,0.014918,0.010801,0.011032,0.004783,0.007296,0.012754,0.000435,0.030234,...,0.021525,0.003022,0.0,0.003424,0.002900,0.019388,1.946248e-47,0.001622,0.002305,0.007124
2020-11-21,0.003564,0.024780,0.014462,0.010692,0.010633,0.002392,0.006275,0.011201,0.000487,0.029162,...,0.020479,0.003077,0.0,0.003823,0.001833,0.020498,9.731239e-48,0.001529,0.001757,0.006413
2020-11-22,0.004063,0.021164,0.014605,0.009293,0.008092,0.001196,0.004669,0.011148,0.000477,0.025380,...,0.025029,0.002840,0.0,0.003923,0.001299,0.020142,4.865620e-48,0.002198,0.001741,0.005823


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.005457,0.018910,0.006823,0.000208,0.004928,0.002604,0.006735,0.015968,6.593731e-11,0.044382,...,0.005895,0.002312,NaN,0.003390,6.579441e-26,0.014131,0.0,0.000620,0.001198,0.005077
2020-11-21,0.006943,0.019128,0.007660,0.000104,0.005458,0.001302,0.004890,0.016142,3.296865e-11,0.045572,...,0.002947,0.001156,NaN,0.003427,3.289721e-26,0.018617,0.0,0.000310,0.000599,0.002538
2020-11-22,0.007053,0.019783,0.007600,0.000052,0.004217,0.000651,0.003800,0.013509,1.648433e-11,0.035673,...,0.015966,0.000578,NaN,0.002864,1.644860e-26,0.022212,0.0,0.000155,0.001704,0.006930


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-20,0.001262,0.029962,0.013960,0.019444,0.022579,0.000207,0.008688,0.045128,0.000301,0.046755,...,0.019553,0.003204,NaN,0.003602,0.004656,0.010412,1.140380e-49,0.005112,0.002880,0.003190
2020-11-21,0.001366,0.028730,0.013306,0.014589,0.022249,0.004041,0.008264,0.033996,0.000170,0.043405,...,0.020208,0.003121,NaN,0.003444,0.002328,0.009560,5.701898e-50,0.004997,0.003036,0.001960
2020-11-22,0.007910,0.026422,0.012994,0.013722,0.016143,0.002020,0.007687,0.023790,0.000183,0.041700,...,0.018113,0.002784,NaN,0.001722,0.001164,0.010244,2.850949e-50,0.004927,0.002778,0.001891


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,0.010631,0.019742,NaN,0.014005,0.014380,0.011524,0.031194,0.022224,0.016810,3.763165e-76,...,0.010885,0.011462,0.024743,0.040676,0.007352,0.010894,0.014784,0.028791,0.020806,0.036723
2020-11-21,0.010428,0.022680,NaN,0.013238,0.013902,0.012959,0.031806,0.011112,0.016599,1.881582e-76,...,0.012011,0.010948,0.022323,0.032914,0.006712,0.010950,0.013544,0.028922,0.020027,0.023541
2020-11-22,0.009111,0.021423,NaN,0.013952,0.011651,0.010590,0.025973,0.005556,0.016074,9.407911e-77,...,0.012836,0.009451,0.020349,0.028160,0.007716,0.010383,0.006772,0.025573,0.015412,0.025616


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-20      0.009944  0.011593  0.005121  0.008809  0.011033  0.003872   
2020-11-21      0.008267  0.011581  0.005167  0.010264  0.011249  0.004860   
2020-11-22      0.007984  0.009972  0.003880  0.053394  0.009098  0.003157   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2020-11-20      0.006691  0.011847  0.013146  0.010328  ...  0.01751   
2020-11-21      0.008703  0.008298  0.008377  0.015592  ...  0.01005   
2020-11-22      0.004793  0.007525  0.005986  0.010860  ...  0.00804   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-20      0.011666  0.051449  0.052612   0.055037  0.026257  0.035886   
2020-11-21      0.009379  0.026667  0.054959   0.036757  0.022110  0.021858   
2020-11-22      0.017013  0.031202  0.034255   0.033981  0.015099  0.021473   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-20           -1.0  0.054001  0.018434  
2020-11-21           -1.0  0.040794  0.011964  
2020-11-22           -1.0  0.048920  0.014179  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-20,0.012917,0.003986,NaN,0.005345,0.010725,0.004868,0.013664,0.004152,0.003517,NaN,...,0.017180,0.009077,0.019412,0.014483,5.284433e-07,0.005802,0.006124,0.022273,0.026061,0.039753
2020-11-21,0.007328,0.006943,NaN,0.005006,0.008809,0.003292,0.014482,0.002076,0.001758,NaN,...,0.009661,0.008201,0.018762,0.015306,2.642216e-07,0.006224,0.003062,0.026004,0.021442,0.019877
2020-11-22,0.003664,0.008374,NaN,0.003045,0.008684,0.003092,0.010650,0.001038,0.002220,NaN,...,0.011361,0.006732,0.013193,0.007653,1.321108e-07,0.003112,0.001531,0.016041,0.011835,0.009938


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-20      0.015802  0.000009  0.013889  0.007388  0.003978  0.000083   
2020-11-21      0.007901  0.000004  0.006944 -0.024084  0.016275  0.000041   
2020-11-22      0.003951  0.000002  0.003472 -0.012042  0.008137  0.000021   

Province_State                                                  ...   Wyoming  \
Admin2                Butler   Calhoun      Chambers  Cherokee  ...      Park   
2020-11-20      7.450581e-10  0.080472  1.644462e-07  0.133333  ...  0.002604   
2020-11-21      3.725290e-10  0.040236  8.222308e-08  0.066667  ...  0.001302   
2020-11-22      1.862645e-10  0.020118  4.111154e-08  0.033333  ...  0.000651   

Province_State                                                        \
Admin2            Platte  Sheridan Sublette Sweetwater         Teton   
2020-11-20      0.001958  0.035632      0.0   0.125008  3.814697e-06   
2020-11-21      0.000979  0.017816      0.0   0.062504  1.907349e-06   
2020-11-22      0.000489  0.008908      0.0   0.031252  9.536743e-07   

Province_State                                               
Admin2             Uinta Unassigned      Washakie    Weston  
2020-11-20      0.000041  -0.972376  3.031649e-13 -0.533333  
2020-11-21      0.000020  -0.972376  1.515825e-13 -0.533333  
2020-11-22      0.000010  -0.972376  7.579123e-14 -0.533333  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201122,AK,26587,NaN,899774,NaN,totalTestsViral,926361,133.0,612.0,...,926361,1,0,597df917af4cf6d52bb240230523f16f37ab083e,0,0,0,0,0,NaN
1,20201122,AL,232506,37921.0,1330801,NaN,totalTestsViral,1525386,1332.0,23449.0,...,1563307,0,0,547eed4fa2ca857a0b99b1bc8989877ce9200b22,0,0,0,0,0,NaN
2,20201122,AR,145173,16680.0,1466454,NaN,totalTestsViral,1594947,962.0,8472.0,...,1611627,20,57,2be6238eabc4a82dec8189ebbc72e50f0c4398e0,0,0,0,0,0,NaN
3,20201122,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,4bdf6f37b45e0e265de85585ad5dceda0873a230,0,0,0,0,0,NaN
4,20201122,AZ,299665,8622.0,1813309,NaN,totalTestsPeopleViral,2104352,1932.0,24412.0,...,2112974,7,231,8453a49b47d5b44fc7cb0db4c2f4e2854276eebb,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201122,AK,26587.0,NaN,899774.0,NaN,totalTestsViral,926361.0,133.0,612.0,...,926361,1,0,597df917af4cf6d52bb240230523f16f37ab083e,0,0,0,0,0,NaN
1,20201122,AL,232506.0,37921.0,1330801.0,NaN,totalTestsViral,1525386.0,1332.0,23449.0,...,1563307,0,0,547eed4fa2ca857a0b99b1bc8989877ce9200b22,0,0,0,0,0,NaN
2,20201122,AR,145173.0,16680.0,1466454.0,NaN,totalTestsViral,1594947.0,962.0,8472.0,...,1611627,20,57,2be6238eabc4a82dec8189ebbc72e50f0c4398e0,0,0,0,0,0,NaN
3,20201122,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,4bdf6f37b45e0e265de85585ad5dceda0873a230,0,0,0,0,0,NaN
4,20201122,AZ,299665.0,8622.0,1813309.0,NaN,totalTestsPeopleViral,2104352.0,1932.0,24412.0,...,2112974,7,231,8453a49b47d5b44fc7cb0db4c2f4e2854276eebb,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-22,AK,26587,NaN,899774,NaN,totalTestsViral,926361,133.0,612.0,NaN,...,926361,1,0,597df917af4cf6d52bb240230523f16f37ab083e,0,0,0,0,0,NaN
2020-11-22,AL,232506,37921.0,1330801,NaN,totalTestsViral,1525386,1332.0,23449.0,NaN,...,1563307,0,0,547eed4fa2ca857a0b99b1bc8989877ce9200b22,0,0,0,0,0,NaN
2020-11-22,AR,145173,16680.0,1466454,NaN,totalTestsViral,1594947,962.0,8472.0,371.0,...,1611627,20,57,2be6238eabc4a82dec8189ebbc72e50f0c4398e0,0,0,0,0,0,NaN
2020-11-22,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,4bdf6f37b45e0e265de85585ad5dceda0873a230,0,0,0,0,0,NaN
2020-11-22,AZ,299665,8622.0,1813309,NaN,totalTestsPeopleViral,2104352,1932.0,24412.0,438.0,...,2112974,7,231,8453a49b47d5b44fc7cb0db4c2f4e2854276eebb,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-11-22,AK,26587.0,NaN,899774.0,NaN,totalTestsViral,926361.0,133.0,612.0,NaN,...,926361,1,0,597df917af4cf6d52bb240230523f16f37ab083e,0,0,0,0,0,NaN
2020-11-22,AL,232506.0,37921.0,1330801.0,NaN,totalTestsViral,1525386.0,1332.0,23449.0,NaN,...,1563307,0,0,547eed4fa2ca857a0b99b1bc8989877ce9200b22,0,0,0,0,0,NaN
2020-11-22,AR,145173.0,16680.0,1466454.0,NaN,totalTestsViral,1594947.0,962.0,8472.0,371.0,...,1611627,20,57,2be6238eabc4a82dec8189ebbc72e50f0c4398e0,0,0,0,0,0,NaN
2020-11-22,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,4bdf6f37b45e0e265de85585ad5dceda0873a230,0,0,0,0,0,NaN
2020-11-22,AZ,299665.0,8622.0,1813309.0,NaN,totalTestsPeopleViral,2104352.0,1932.0,24412.0,438.0,...,2112974,7,231,8453a49b47d5b44fc7cb0db4c2f4e2854276eebb,0,0,0,0,0,NaN
